# Klasifikasi Limbah dengan Convolutional Neural Network (CNN)
Pada tugas ini, kita akan menggunakan **RealWaste Dataset** yang dapat diakses melalui tautan berikut:\
🔗 https://www.kaggle.com/datasets/joebeachcapital/realwaste/data

Dataset ini berisi gambar limbah nyata dari lingkungan, dengan berbagai kategori jenis limbah. 
Tujuan utama dari tugas ini adalah membangun **model CNN (Convolutional Neural Network)** untuk mengklasifikasikan jenis limbah berdasarkan gambar.

Langkah-langkah yang dilakukan adalah:
1. Persiapan Dataset & Eksplorasi Awal
2. Preprocessing Data (Resize, Augmentasi, Normalisasi)
3. Membangun Model CNN
4. Training dan Validasi Model
5. Evaluasi dan Visualisasi
6. Analisis dan Kesimpulan

# 1. Import Library

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB3
import warnings
warnings.filterwarnings('ignore')

# 2. Load Dataset & Explorasi Awal

In [ ]:
# Set random seed untuk reprodusibilitas
np.random.seed(42)
tf.random.set_seed(42)

# ==================== 1. PERSIAPAN DATASET & EKSPLORASI AWAL ====================
print("="*60)
print("1. PERSIAPAN DATASET & EKSPLORASI AWAL")
print("="*60)

# Path dataset
train_dir = '/content/drive/MyDrive/Colab-Notebooks/sem3/CNN/train/'
test_dir = '/content/drive/MyDrive/Colab-Notebooks/sem3/CNN/test/'

Found 4752 files belonging to 9 classes.
Using 3802 files for training.
Found 4752 files belonging to 9 classes.
Using 950 files for validation.


In [ ]:
# Fungsi untuk menghitung jumlah gambar per kategori
def count_images_per_class(directory):
    class_counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            class_counts[class_name] = count
    return class_counts

# Hitung distribusi data training
train_counts = count_images_per_class(train_dir)
print("\n📊 Distribusi Data Training:")
for class_name, count in sorted(train_counts.items()):
    print(f"  {class_name}: {count} images")

total_train = sum(train_counts.values())
print(f"\n✅ Total gambar training: {total_train}")

# Visualisasi distribusi kelas
plt.figure(figsize=(12, 6))
classes = list(train_counts.keys())
counts = list(train_counts.values())
bars = plt.bar(classes, counts, color='steelblue', edgecolor='black')
plt.xlabel('Kategori Limbah', fontsize=12, fontweight='bold')
plt.ylabel('Jumlah Gambar', fontsize=12, fontweight='bold')
plt.title('Distribusi Data Training per Kategori', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Tambahkan nilai di atas bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}',
             ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# Visualisasi contoh gambar dari tiap kategori
print("\n🖼️ Visualisasi Contoh Gambar per Kategori:")
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.ravel()

for idx, class_name in enumerate(sorted(classes)):
    class_path = os.path.join(train_dir, class_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if images:
        img_path = os.path.join(class_path, images[0])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[idx].imshow(img)
        axes[idx].set_title(f'{class_name}\n({train_counts[class_name]} images)', fontsize=11, fontweight='bold')
        axes[idx].axis('off')

plt.tight_layout()
plt.show()

# 3. Preprocessing & Augmentasi Data

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50

# Data augmentation untuk training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 80% train, 20% validation
)

# Data generator untuk validation (hanya rescaling)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Load validation data
validation_generator = val_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

# Simpan class indices untuk prediksi nanti
class_indices = train_generator.class_indices
class_names = {v: k for k, v in class_indices.items()}

print(f"\n✅ Training samples: {train_generator.samples}")
print(f"✅ Validation samples: {validation_generator.samples}")
print(f"\n📋 Class Mapping:")
for idx, name in class_names.items():
    print(f"  {idx}: {name}")

# 4. Membangun Model CNN

In [ ]:
def create_model(num_classes):
    # Menggunakan EfficientNetB3 sebagai base model (transfer learning)
    base_model = EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    # Freeze base model layers untuk fine-tuning bertahap
    base_model.trainable = False

    # Build model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model, base_model

# Create model
num_classes = len(class_indices)
model, base_model = create_model(num_classes)

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("\n📐 Arsitektur Model:")
model.summary()

# 5. Training & Validasi Model

In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# Training fase 1: Train dengan base model frozen
print("\n🚀 Training Fase 1: Base Model Frozen")
history1 = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25,
    callbacks=callbacks,
    verbose=1
)

# Fine-tuning: Unfreeze beberapa layer terakhir dari base model
print("\n🔥 Training Fase 2: Fine-tuning (Unfreeze base model)")
base_model.trainable = True

# Freeze layer awal, hanya train layer akhir
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Recompile dengan learning rate lebih kecil
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

# Continue training
history2 = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25,
    callbacks=callbacks,
    verbose=1
)

# Gabungkan history dari kedua fase
history = {
    'loss': history1.history['loss'] + history2.history['loss'],
    'val_loss': history1.history['val_loss'] + history2.history['val_loss'],
    'accuracy': history1.history['accuracy'] + history2.history['accuracy'],
    'val_accuracy': history1.history['val_accuracy'] + history2.history['val_accuracy']
}

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.1888 - loss: 2.1187 - val_accuracy: 0.2484 - val_loss: 2.0387
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.2438 - loss: 1.9556 - val_accuracy: 0.2832 - val_loss: 2.0875
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.3077 - loss: 1.8574 - val_accuracy: 0.3937 - val_loss: 1.6076
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.3588 - loss: 1.7407 - val_accuracy: 0.4516 - val_loss: 1.6239
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.3958 - loss: 1.6862 - val_accuracy: 0.4789 - val_loss: 1.4737
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accuracy: 0.3940 - loss: 1.6373 - val_accuracy: 0.4716 - val_loss: 1.4620
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.4237 - loss: 1.5910 - val_accuracy: 0.4726 - val_loss: 1.4560
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.4116 - loss: 1.5831 - val_accu

# 6. Evaluasi & Visualisasi

In [ ]:
# Load best model
model.load_weights('best_model.h5')

# Evaluasi pada validation set
print("\n📊 Evaluasi pada Validation Set:")
val_loss, val_accuracy, val_precision, val_recall = model.evaluate(validation_generator)
val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall)

print(f"\n✅ Validation Accuracy: {val_accuracy*100:.2f}%")
print(f"✅ Validation Precision: {val_precision*100:.2f}%")
print(f"✅ Validation Recall: {val_recall*100:.2f}%")
print(f"✅ Validation F1-Score: {val_f1*100:.2f}%")

# Prediksi pada validation set untuk confusion matrix
validation_generator.reset()
y_pred = model.predict(validation_generator, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = validation_generator.classes

# Classification report
print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred_classes,
                          target_names=list(class_names.values()),
                          digits=4))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)

30/30 ━━━━━━━━━━━━━━━━━━━━ 11s 349ms/step - accuracy: 0.4505 - loss: 1.5041
Test Loss: 1.5040987730026245
Test Accuracy: 0.45052632689476013


In [ ]:
# Fungsi untuk prediksi gambar individual
def predict_image(img_path, model, class_names):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img, verbose=0)
    predicted_class = np.argmax(prediction[0])
    confidence = prediction[0][predicted_class]

    return class_names[predicted_class], confidence

# Ambil semua file gambar dari folder test
test_images = []
for file in os.listdir(test_dir):
    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
        test_images.append(file)

print(f"\n📁 Total gambar test: {len(test_images)}")

# Prediksi semua gambar test
results = []
print("\n🔮 Melakukan prediksi...")

for img_file in test_images:
    img_path = os.path.join(test_dir, img_file)
    pred_label, confidence = predict_image(img_path, model, class_names)
    results.append({
        'filename': img_file,
        'label': pred_label,
        'confidence': confidence
    })

# Buat DataFrame
df_results = pd.DataFrame(results)

# Save ke CSV (format yang diminta)
output_csv = 'predictions.csv'
df_results[['filename', 'label']].to_csv(output_csv, index=False)

print(f"\n✅ Prediksi selesai! File disimpan di: {output_csv}")
print(f"\n📊 Preview hasil prediksi:")
print(df_results.head(10))

# Statistik prediksi
print(f"\n📈 Distribusi Prediksi Test Set:")
pred_counts = df_results['label'].value_counts()
for label, count in pred_counts.items():
    print(f"  {label}: {count} images")

# Visualisasi distribusi prediksi test
plt.figure(figsize=(12, 6))
pred_counts.plot(kind='bar', color='coral', edgecolor='black')
plt.xlabel('Kategori Limbah', fontsize=12, fontweight='bold')
plt.ylabel('Jumlah Prediksi', fontsize=12, fontweight='bold')
plt.title('Distribusi Prediksi pada Data Test', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Visualisasi beberapa contoh prediksi dari test set
print("\n🖼️ Visualisasi Contoh Prediksi Test Set:")
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.ravel()

for idx in range(min(9, len(test_images))):
    img_path = os.path.join(test_dir, test_images[idx])
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    pred_label = df_results.iloc[idx]['label']
    confidence = df_results.iloc[idx]['confidence']

    axes[idx].imshow(img)
    axes[idx].set_title(f'{test_images[idx]}\nPrediksi: {pred_label}\nConfidence: {confidence*100:.1f}%',
                       fontsize=9, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# 7. Analisis & Kesimpulan